In [1]:
# FILE_A = '../data/IDS2012/TestbedTueJun15-1Flows.json'
# FILE_B = '../data/IDS2012/TestbedTueJun15-2Flows.json'
# FILE_C = '../data/IDS2012/TestbedTueJun15-3Flows.json'
FILE = 'TestbedTueJun15-2Flows.json'

In [2]:
import os
import pandas as pd
import json
import glob
import gc
from datetime import datetime
from collections import defaultdict
from sklearn import ensemble
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

DATA_PATH = os.path.join('../data', 'IDS2012')
# ATTACK_COUNT = 0

def load_data_set(data_path):
    json_path = os.path.join(data_path, FILE)
    dataset = []
    for file in glob.glob(json_path):
        filename = file.split('/')[-1].split('.')[0]
        json_file = None
        temp = None
        try:
            json_file = open(file, 'r')
            temp = json.load(json_file).get('dataroot').get(filename)

            # Clean dataset
            for item in temp:
                # Delete unnecessary features
                del item['appName']
                del item['sourcePayloadAsBase64']
                del item['sourcePayloadAsUTF']
                del item['destinationPayloadAsBase64']
                del item['destinationPayloadAsUTF']
                del item['sourceTCPFlagsDescription']
                del item['destinationTCPFlagsDescription']

                # Count total number of IP address occurences
                ip_counts['source'][item['source']] += 1
                ip_counts['destination'][item['destination']] += 1

                # Convert into more appropriate features
                item['direction'] = convert_direction(item['direction'])
                item['duration'] = calculate_duration(
                    item.pop('startDateTime'), item.pop('stopDateTime'))
                tag_num = convert_class(item['Tag'])
                item['Tag'] = tag_num

                # Delete features for prototype
                del item['totalSourceBytes']
                del item['totalDestinationBytes']
                del item['totalDestinationPackets']
                del item['totalSourcePackets']
                del item['direction']

            dataset += temp
        finally:
            if json_file is not None:
                json_file.close()
                json_file = None
                gc.collect()
    return dataset


def generate_arr(dataset, classification):
    classification_arr = dataset[classification].values
    del dataset[classification]
    dataset_arr = dataset.values

    return dataset_arr, classification_arr


def convert_class(x):
    return int(x != 'Normal')


def convert_direction(x):
    return int(x != 'L2R')


def calculate_duration(start, stop):
    dt = datetime.strptime(stop, '%Y-%m-%dT%H:%M:%S') - datetime.strptime(
        start, '%Y-%m-%dT%H:%M:%S')
    return dt.total_seconds()


# Load training data
ip_counts = {'source': defaultdict(int), 'destination': defaultdict(int)}
flows = load_data_set(DATA_PATH)

for flow in flows:
    flow['source_ip_count'] = ip_counts['source'][flow.pop('source')]
    flow['destination_ip_count'] = ip_counts['destination'][flow.pop(
        'destination')]

temp = pd.DataFrame.from_dict(flows)
data = pd.get_dummies(temp, prefix=['protocol'], columns=['protocolName'])
# del data['sensorInterfaceId']
# del data['startTime']

# train_len = 1500000
# packets = data[:train_len]
# test_packets = data[train_len:]

# # Generate arrays
# packets_arr, classification_arr = generate_arr(packets, 'Tag')
# test_packets_arr, test_classification_arr = generate_arr(test_packets, 'Tag')

# # Train classifier
# clf = ensemble.AdaBoostClassifier()
# clf.fit(packets_arr, classification_arr)

# # Test classifier
# pred = clf.predict(test_packets_arr)

# # Check accuracy
# accuracy = accuracy_score(test_classification_arr, pred)
# print 'Accuracy:', accuracy

# # Save model
# joblib.dump(clf, 'adaboost-ids.pkl')


In [3]:
data.head()

,Tag,destinationPort,destination_ip_count,duration,sourcePort,source_ip_count,protocol_icmp_ip,protocol_tcp_ip,protocol_udp_ip
0,0,80,1,598.0,3489,19701,0,1,0
1,0,80,269,505.0,2141,2551,0,1,0
2,0,80,1,254.0,2022,7802,0,1,0
3,0,80,2152,259.0,3925,14263,0,1,0
4,0,80,2152,258.0,3933,14263,0,1,0


In [4]:
ip_counts['destination']['192.168.1.244']

0

In [5]:
(data['Tag'] == 1).sum()

1